In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()

## Dependecies

In [2]:
import sys
import ee

from functools import partial


ACOLITE_PATH = r'C:\Users\sergi\Documents\repos\gee_acolite\src'

sys.path.append(ACOLITE_PATH)

import ACOLITE

## Sentinel-2 ACOLITE-Fixed

In [3]:
for (bounds, start, end, tile) in [
                                # ([-7.849388, 36.970241, -7.724075, 37.032705], '2018-10-29', '2018-10-30', '29SPA'),
                                # ([-7.849388, 36.970241, -7.724075, 37.032705], '2019-10-02', '2019-10-03', '29SPA'),
                                # ([-7.849388, 36.970241, -7.724075, 37.032705], '2020-10-18', '2020-10-19', '29SPA'),

                                # ([-64.028848, -63.694804, -58.117281, -62.744728], '2016-11-13', '2016-11-14', None),

                                # ([-9.68713, 35.538631, 4.949517, 43.978157], '2024-07-01', '2024-07-02', None),
                                
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2016-11-13', '2016-11-14', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-01-19', '2020-01-20', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-12-27', '2020-12-28', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2021-11-22', '2021-11-23', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2022-03-29', '2022-03-30', '21EUL'),
                                # # ([-60.827179, -63.048731, -60.420685, -62.867057], '2022-08-16', '2022-08-17', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-03-17', '2023-03-18', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-09-10', '2023-09-11', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2024-10-24', '2024-10-25', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2025-01-25', '2025-01-26', '21EUL'),
                                ]:
    
    print(f'Tile {tile}, L1 to L2 via Fixed DSF AC {start=}, {end=}')

    # pre processing
    roi = ee.Geometry.Rectangle(bounds)
    
    tolerance = 10
    mask_clouds_by_probability = partial(ACOLITE.mask_clouds_by_probability, threshold = tolerance)
    sentinel2_l1 = ACOLITE.search_with_clouds(roi, start, end, tile = tile).map(ACOLITE.add_cloud_probability).map(mask_clouds_by_probability)
    print(f'Collection size {sentinel2_l1.size().getInfo()}')

    sentinel2_l1r_rrs = sentinel2_l1.map(ACOLITE.select_sentinel2_bands).map(ACOLITE.to_rrs) # Digital Number to rrs
    sentinel2_list = sentinel2_l1r_rrs.toList(sentinel2_l1r_rrs.size()) # Collection to List for iteration
    # pre processing

    # during
    sentinel2_l2r_rrs = ee.ImageCollection.fromImages( [ ACOLITE.dsf(ee.Image(sentinel2_list.get(i)))[0] for i in range(sentinel2_l1r_rrs.size().getInfo()) ] )
    sentinel2_l2r_rrs_deglint = ee.ImageCollection.fromImages( [ ACOLITE.deglint_alternative(*ACOLITE.dsf(ee.Image(sentinel2_list.get(i)))) for i in range(sentinel2_l1r_rrs.size().getInfo()) ] )
    # during

    # post processing
    sentinel2_l2w_rrs = sentinel2_l2r_rrs.map(ACOLITE.mask_water_by_SWIR1)
    sentinel2_l2w_rrs_deglint = sentinel2_l2r_rrs_deglint.map(ACOLITE.mask_water_by_SWIR1)
    # post processing
    
    for product in [ACOLITE.add_zgreen, ACOLITE.add_zred]:
        sentinel2_l2w_rrs = sentinel2_l2w_rrs.map(product)
        sentinel2_l2w_rrs_deglint = sentinel2_l2w_rrs_deglint.map(product)
    
    # geemap.download_ee_image(sentinel2_l2w_rrs.first().select(['zgreen', 'zred']), 
    #                      rf"C:\Users\sergi\Documents\repos\gee_acolite\results\DCI\Version_20231023_Date_{start.replace('-','')}.tif", 
    #                      roi, 'EPSG:32721', scale = 10)
    #                     #  roi, 'EPSG:25829', scale = 10)
    # geemap.download_ee_image(sentinel2_l2w_rrs_deglint.first().select(['zgreen', 'zred']), 
    #                      rf"C:\Users\sergi\Documents\repos\gee_acolite\results\DCI\Version_20231023_Deglint_Date_{start.replace('-','')}.tif", 
    #                      roi, 'EPSG:32721', scale = 10)
    #                     #  roi, 'EPSG:25829', scale = 10)

Tile None, L1 to L2 via Fixed DSF AC start='2016-11-13', end='2016-11-14'
Collection size 18


In [4]:
import geemap.colormaps as cm

rgb_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': 1.4
}

zred_params = {
    'palette': cm.palettes.viridis_r,
    'min': 1,
    'max': 1.3,
}


deception_land = geemap.shp_to_ee(r'C:\Users\sergi\Documents\repos\bathypy\data\shapefiles\deception\deception.shp')
mask = ee.Image.constant(1).clip(deception_land).mask().Not()

Map = geemap.Map()
Map.add_basemap('Esri.WorldImagery', show=False)
Map.addLayerControl()
Map.center_object(roi)
Map.addLayer(sentinel2_l2r_rrs.mosaic(), rgb_params, f"Sentinel2 L2R")
Map.addLayer(sentinel2_l2w_rrs.mosaic().updateMask(mask).select('zred'), zred_params, f"Sentinel2 L2W")
Map.addLayer(roi, {}, f"ROI")
Map.add_colorbar(zred_params, label="Z red (DII)", layer_name="zred")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…